In [1]:
# Giovani Nascimento Pereira - 168609
# Carlos Augusto Figueiredo Feire de Carvalho - 165684

import numpy
import sys
from PIL import Image, ImageFilter
from numpy import genfromtxt
from sklearn.cluster import KMeans, MiniBatchKMeans, AffinityPropagation
from sklearn.metrics import silhouette_score
import csv

nTotal = 19924
#nTotal = 1000
nFeatures = 2209
nTreino = int(0.9 * nTotal)
nTeste  = 1992

#Atualizar com o caminho do csv no seu diretorio
csvPath = "./dataset/csv.noUp/data.csv"


# Pegando os dados
print('-----------Pegando os dados----------')
reader = csv.reader(open(csvPath, "r"), delimiter=",")
med = list(reader)
data= numpy.array(med).astype("float")

# print(data.shape)
# Dimensao dos dados completos: (19924, 2209) -> as expected
# 19924 documentos
# 2209 features

# Separa em teste
print('-----------Montando Teste----------')
X_teste = numpy.ones((nTeste, nFeatures + 1))
for num in range(0, nTeste):
	X_teste[num, 1:(nFeatures+1)] = data[num, 0:nFeatures]

# Separa em treino
print('-----------Montando Treino----------')
X_treino = numpy.ones((nTreino, nFeatures + 1))
for num in range(0, nTreino):
	X_treino[num, 1:(nFeatures+1)] = data[num + nTeste, 0:nFeatures]

-----------Pegando os dados----------
-----------Montando Teste----------
-----------Montando Treino----------


In [ ]:
# Define modelo do KMeans - parâmetros:
# KMeans(n_clusters=8,
#		init=’k-means++’,
#		n_init=10,
#		max_iter=300,
#		tol=0.0001,
#		precompute_distances=’auto’,
#		verbose=0, random_state=None,
#		copy_x=True, n_jobs=1, algorithm=’auto’)

kmeans = KMeans(random_state = 1)

#  Primeira parte do treinamento, treinamos usando kMeans e incrementando o
# numero de clusters a cada iteração
#
print('Primeira parte\n')
maior = 0
id_maior = 2

for i in range (2, 101):
	# altera o numero de clusters
	kmeans = KMeans(n_clusters=i)
	kmeans.fit(X_treino)
	# predição dos labels para o dataset de teste
	label_teste = kmeans.predict(X_teste)
	print('Numero de clusters: ', i)
	# calcula e imprime os silhouette_score desse modelo com esse numero de clusters
	silhouette_score_new = silhouette_score(X_teste, label_teste)
	print(silhouette_score_new, '\n')
	# confere qual o numero de clusters que deu melhor resultado
	if(silhouette_score_new > maior):
		maior = silhouette_score_new
		id_maior = i

print('Melhor cluster: ', id_maior, '\t com silhouette_score de :', maior)

# Fim do primeiro teste
#
#

#  Segundo teste, pegamos o melhor resultado do primeiro cluster e mudamos
# o init, que por padrão era kMeans++, p/ random
#

kmeans = KMeans(random_state = 1, init = 'random', n_clusters = id_maior)
kmeans.fit(X_treino)
label_teste = kmeans.predict(X_teste)
print('Mudando o método de inicialização do centroides para random: ')
silhouette_score_new = silhouette_score(X_teste, label_teste)
print(silhouette_score_new, '\n')

# Fim do segundo teste
#
#

Primeira parte

Numero de clusters:  2
0.0116262133258 

Numero de clusters:  3
0.0149159578913 

Numero de clusters:  4
0.0165326227568 

Numero de clusters:  5
0.017607085408 

Numero de clusters:  6
0.0186398209769 

Numero de clusters:  7
0.0200922399365 

Numero de clusters:  8
0.0209727407117 

Numero de clusters:  9
0.0228546156829 

Numero de clusters:  10
0.0248217447888 

Numero de clusters:  11
0.0247415492084 

Numero de clusters:  12
0.0286751809925 



In [2]:
# Fazer um teste com mini-batch k-means
#
#

miniKmeans = MiniBatchKMeans(n_clusters = 100)
miniKmeans.fit(X_treino)
label_mini = miniKmeans.predict(X_teste)
print('Mudando para MiniBatchKMeans temos: ', silhouette_score(X_teste, label_mini), '\n')

#
#
#

Mudando para MiniBatchKMeans temos:  0.0527491216912 



In [ ]:
# Fazer um teste com AffinityPropagation
# http://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html
#
#

affinityPropagation = AffinityPropagation()
affinityPropagation.fit(X_treino)
label_affinity = AffinityPropagation.predict(X_teste)
print('Mudando para AffinityPropagation temos: ', silhouette_score(X_teste, label_affinity), '\n')

#
#
#
#